In [50]:
import tensorflow as tf
import numpy as np
tf.__version__
data_eval_path = "/home/perm/example/learn/data/eval_db.tfrecords"
data_test_path = "/home/perm/example/learn/data/test_db.tfrecords"
data_train_path = "/home/perm/example/learn/data/train_db.tfrecords"
group_regressor_path = "/home/perm/example/learn/data/group_regressor/eval_db.tfrecords"
filenames  =  [group_regressor_path]


In [51]:
raw_dataset = tf.data.TFRecordDataset(filenames)
feature_schema = raw_dataset.element_spec
print(feature_schema)


TensorSpec(shape=(), dtype=tf.string, name=None)


In [52]:
raw_dataset = tf.data.TFRecordDataset(filenames)
for raw_record in raw_dataset.take(2):  # Adjust number of records to peek
    print(repr(raw_record))
    try:
        example = tf.train.Example()
        example.ParseFromString(raw_record.numpy())
        print(example)
    except tf.errors.DataLossError as e:
        print("DataLossError encountered:", e)
        continue

DataLossError: corrupted record at 0

In [69]:
#Eager tensor: operations are run one by one, similar to regular Python code
def _bytes_feature(value):
    """Returns a bytes_list from a string / byte."""
    if isinstance(value, type(tf.constant(0))):
        value = value.numpy()  ## BytesList won't unpack a string from an EagerTensor
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))
def _float_features(value):
    """Returns a float_list from a float / double"""
    return tf.train.Feature(float_list=tf.train.FloatList(value=[value]))

def _int64_feature(value):
    """Returns an int64_list from a bool / enum / int / uint."""
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value])) 

def serialize_example(feature0, feature1):
    my_feature = {
           'feature0': _int64_feature(feature0),
           'feature1': _float_features(feature1) 
                       
    }
    #Creating feature message 
    proto_buff_instance = tf.train.Example(features=tf.train.Features(feature=my_feature))
    return proto_buff_instance.SerializeToString()

def tfrecord_writer(data, filename):
    with tf.io.TFRecordWriter(filename) as writer:
        for feature0, feature1 in data:
            example = serialize_example(feature0, feature1)
            writer.write(example)
    print('Record Written Successfully to path: ',filename)



In [70]:
#Writing TFRecord file
#sample data first with int and float 
data = [(i, 0.1*i) for i in range(5)]
# [(0,0.1 ), (1, 0.2)]
filename= '/home/perm/example/learn/data/my_sample_record.tfrecord'
tfrecord_writer(data, filename)

Record Written Successfully to path:  /home/perm/example/learn/data/my_sample_record.tfrecord


In [55]:
type(tf.constant(0))

tensorflow.python.framework.ops.EagerTensor

In [76]:
#Reading file from tf record 
def _parse_function(proto):
    features = {
        'feature0' : tf.io.FixedLenFeature([], tf.int64),
        'feature1' : tf.io.FixedLenFeature([], tf.float32)
    }
    return tf.io.parse_single_example(proto, features)

def tfrecord_reader(filename):
    dataset = tf.data.TFRecordDataset(filename)
    dataset = dataset.map(_parse_function)
    for features in dataset:
        print(f"Feature 0  {features['feature0'].numpy()}  Feature1: {features['feature1'].numpy()}")

In [77]:
tfrecord_reader('/home/perm/example/learn/data/my_sample_record.tfrecord')

Feature 0  0  Feature1: 0.0
Feature 0  1  Feature1: 0.10000000149011612
Feature 0  2  Feature1: 0.20000000298023224
Feature 0  3  Feature1: 0.30000001192092896
Feature 0  4  Feature1: 0.4000000059604645


In [85]:
dataset = tf.data.TFRecordDataset('/home/perm/example/learn/data/my_sample_record.tfrecord')
#dataset
# dataset.element_spec
for raw_data in dataset.take(1):
    # print(repr(raw_data))
    example = tf.train.Example()
    example.ParseFromString(raw_data.numpy())
    print(example)

features {
  feature {
    key: "feature0"
    value {
      int64_list {
        value: 0
      }
    }
  }
  feature {
    key: "feature1"
    value {
      float_list {
        value: 0.0
      }
    }
  }
}



: 